In [350]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

 # device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device: ', device)

df = pd.read_csv('./output/colors_2024-03-22T18-04-05.csv')


Using device:  cpu


In [351]:
df.info()
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    1080 non-null   object
 1   r       1080 non-null   int64 
 2   g       1080 non-null   int64 
 3   b       1080 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 33.9+ KB


,word,r,g,b
0,A,113,221,120
1,B,53,191,116
2,C,181,61,238
3,D,84,179,60
4,E,22,64,85
...,...,...,...,...
1075,castle,181,61,238
1076,broad,53,191,116
1077,long,113,58,208
1078,could,181,61,238


In [352]:
# turn loaded dataframe into torch tensors
from word_encoding import WordEncoder

we = WordEncoder()

inputs_list = we.encode_list(df['word'])
inputs = torch.tensor((len(inputs_list),1,45), dtype=torch.float)
inputs = torch.cat(inputs_list, dim=0)
print(inputs.shape)
torch.save(inputs, 'inputs0.pt')
print(inputs[0].shape)


torch.Size([1080, 45])
torch.Size([45])


In [353]:
outputs = torch.tensor(df.drop(['word'],axis='columns').values)
print(outputs.shape)
print(outputs[0].shape)
torch.save(outputs, 'outputs0.pt')

torch.Size([1080, 3])
torch.Size([3])


In [354]:
# create dataset class to take inputs & outputs
class Data(Dataset):
    # Constructor
    def __init__(self, inputs, outputs):
        self.x = inputs
        self.x = self.x.to(device)
        self.y = outputs
        self.y = self.y.to(device)
        self.len = len(inputs)
    
    # Getter
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    # Get number of samples
    def __len__(self):
        return self.len

In [355]:
# Making Model
class ColorPredictor(torch.nn.Module):
    #Constructor
    def __init__(self):
        super(ColorPredictor, self).__init__()
        self.linear1 = torch.nn.Linear(45, 20, device= device) #length of encoded word vectors
        self.linear2 = torch.nn.Linear(20,10, device= device)
        self.linear3 = torch.nn.Linear(10,3, device=device)
        self.reLU = torch.nn.ReLU()
        
    # Prediction
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.linear1(x)
        out = self.reLU(out)
        out = self.linear2(out)
        out = self.reLU(out)
        out = self.linear3(out)
        return out

In [356]:
# Create model

model = ColorPredictor()
model.to(device)


ColorPredictor(
  (linear1): Linear(in_features=45, out_features=20, bias=True)
  (linear2): Linear(in_features=20, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
  (reLU): ReLU()
)

In [ ]:
# create train/test split
train_size = int(0.8 * len(inputs))
print(train_size)
print(len(inputs)- train_size)
train_data = Data(inputs[:train_size], outputs[:train_size])
test_data = Data(inputs[train_size:], outputs[train_size:])

In [ ]:
# create DataLoaders
train_loader = DataLoader(dataset = train_data, batch_size=2)
test_loader = DataLoader(dataset = test_data, batch_size=2)

In [357]:
# criterion & optimizer
criterion = nn.MSELoss()
#criterion = nn.L1Loss()
adam = torch.optim.Adam(model.parameters())
# adamw = torch.optim.AdamW(model.parameters())

In [ ]:
# train & test loops
import copy
import numpy as np
torch.manual_seed(42)

epochs = 1000

epoch_count = []
loss_values = []
test_loss_values = []
best_mse = 155.8818
best_weights = None


for epoch in range(epochs):
    # Train
    model.train()
    for x,y in train_loader:
        y_pred = model(x)
        loss = criterion(y_pred.float(), y.float())
        adam.zero_grad()
        loss.backward()
        adam.step()
        
        
     # test
    model.eval()
    for x,y in test_loader:
        test_pred = model(x)           
        test_loss = criterion(test_pred, y)
        if test_loss < best_mse:
            best_mse = test_loss
            best_weights = copy.deepcopy(model.state_dict())
            


print(best_mse)    

In [358]:
from pathlib import Path

MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = 'demo_model.pth'
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(model.state_dict(), f=MODEL_SAVE_PATH)

In [359]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

<All keys matched successfully>

In [360]:
predictions = []
actual = []

total_data = Data(inputs, outputs)
total_loader = DataLoader(total_data, batch_size=2)

model.eval()
with torch.inference_mode():
    for x,y in total_loader:
        test_pred = model(x)
        predictions.append(test_pred)
        actual.append(y)
        test_loss = criterion(test_pred, y)


In [361]:
print(predictions[0:10])
print(actual[0:10])

[tensor([[ 0.3243,  0.3142, -0.2966],
        [ 0.3245,  0.3140, -0.2966]]), tensor([[ 0.3246,  0.3139, -0.2966],
        [ 0.3248,  0.3137, -0.2966]]), tensor([[ 0.3249,  0.3135, -0.2966],
        [ 0.3251,  0.3133, -0.2966]]), tensor([[ 0.3252,  0.3131, -0.2966],
        [ 0.3254,  0.3129, -0.2966]]), tensor([[ 0.3255,  0.3128, -0.2966],
        [ 0.3257,  0.3126, -0.2966]]), tensor([[ 0.3258,  0.3124, -0.2966],
        [ 0.3260,  0.3122, -0.2966]]), tensor([[ 0.3261,  0.3120, -0.2966],
        [ 0.3263,  0.3118, -0.2966]]), tensor([[ 0.3264,  0.3117, -0.2965],
        [ 0.3266,  0.3115, -0.2965]]), tensor([[ 0.3268,  0.3113, -0.2965],
        [ 0.3269,  0.3112, -0.2965]]), tensor([[ 0.3271,  0.3110, -0.2965],
        [ 0.3273,  0.3108, -0.2965]])]
[tensor([[113, 221, 120],
        [ 53, 191, 116]]), tensor([[181,  61, 238],
        [ 84, 179,  60]]), tensor([[ 22,  64,  85],
        [ 31, 208, 249]]), tensor([[ 68, 238,  98],
        [239, 217, 198]]), tensor([[141, 120, 127],
     